Réccupération de tout les cours distinct de la base

class pour vectoriser messages (creation d'un nouveau champs)

In [157]:
from mistralai import Mistral
# from dotenv import load_dotenv
# import os
import time
# load_dotenv(dotenv_path='./.env')

def create_client():
    api_key = "5D6qWiGcW0q7sMoNJKnz7gvTQ7VinHue"
    return Mistral(api_key=api_key)

class MistralEmbedder:
    def __init__(self):
        api_key = "5D6qWiGcW0q7sMoNJKnz7gvTQ7VinHue"
        # api_key = os.environ["5D6qWiGcW0q7sMoNJKnz7gvTQ7VinHue"]
        self.client = Mistral(api_key=api_key)
        self.model = "mistral-embed"

   

    import time

class MistralEmbedder:
    def __init__(self, client=Mistral(api_key="5D6qWiGcW0q7sMoNJKnz7gvTQ7VinHue"), model="mistral-embed", batch_size=16):
        self.client = client
        self.model = model
        self.batch_size = batch_size

    
    def embed_messages(self, messages, batch_size=250):
        """
        Vectorise les messages par batchs, ajoute 'vectorized' à chacun.
        Gère les erreurs 429 avec retry.
        """
        total = len(messages)
        print(f"🚀 Traitement de {total} messages par batchs de {batch_size}...")

        for i in range(0, total, batch_size):
            batch = messages[i:i + batch_size]
            texts = []
            valid_msgs = []

            for msg in batch:
                text = msg.get('content','').get('body', '').strip()
                if text:
                    texts.append(text)
                    valid_msgs.append(msg)

            if not texts:
                continue

            retries = 0
            success = False

            while not success and retries < 5:
                try:
                    response = self.client.embeddings.create(
                        inputs=texts,
                        model=self.model
                    )

                    for msg, emb in zip(valid_msgs, response.data):
                        msg['content']['vectorized'] = emb.embedding
                        print(f"✅ {i + 1}/{total} : OK")

                    success = True

                except Exception as e:
                    if "429" in str(e) or "rate limit" in str(e).lower():
                        wait = 10 * (retries + 1)
                        print(f"⚠️ Rate limit — attente {wait}s (essai {retries+1}/5)")
                        time.sleep(wait)
                        retries += 1
                    else:
                        print(f"❌ Erreur batch {i}-{i+batch_size} : {e}")
                        for msg in valid_msgs:
                            msg['vectorized'] = None
                        break

            time.sleep(0.3)  # Pause entre chaque batch (important)

        print("✅ Tous les messages ont été traités.")
        return messages

In [158]:
from pymongo import MongoClient

# Connexion à la base de données MongoDB
client = MongoClient("mongodb://localhost:27017")  # Mets l’URL adaptée à ta base
db = client["mooc"]  # Remplace par le nom de ta base
collection = db["sample"]

# Récupérer les valeurs distinctes de course_id dans content
distinct_course_ids = collection.distinct("content.course_id")

print(distinct_course_ids)


['CNAM/01002/Trimestre_1_2014', 'CNAM/01002S04/session04', 'CNAM/01020S02/session02', 'CNAM/01021/session01', 'CNAM/01023/session01', 'CoR/114001/session01', 'ENSCachan/20013S02/session02', 'ENSDeLyon/14003S03/session03', 'MinesTelecom/04002/Trimestre_1_2014', 'MinesTelecom/04006S04/session04', 'MinesTelecom/04006S06/session06', 'MinesTelecom/04010S03/session03', 'MinesTelecom/04012S03/session03', 'MinesTelecom/04014S03/session03', 'MinesTelecom/04015S02/session02', 'MinesTelecom/04016/session01', 'MinesTelecom/04017/session01', 'MinesTelecom/04017S02/session02', 'MinesTelecom/04018/session01', 'MinesTelecom/04018S02/session02', 'MinesTelecom/04019S02/session02', 'MinesTelecom/04021/session01', 'MinesTelecom/04021S02/session02', 'OBSPM/62002/session01', 'Paris1/16004S02/session02', 'TUMx/AUTONAVx/2T2014', 'UPVD/95001/session01', 'agrocampusouest/40001S03/session03', 'course-v1:AdelaideX+Project101x+2T2016', 'course-v1:BerkeleyX+EE40LX+2T2015', 'course-v1:CNAM+01001+session05', 'course-

In [159]:
from pymongo import MongoClient

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
filter={}
project={
    '_id': 0,
    'content.id': 1,
    'content.course_id': 1, 
    'content.title': 1, 
    'content.body': 1
}
maxTimeMS=60000

result = client['mooc']['sample'].find(
  filter=filter,
  projection=project,
  max_time_ms=maxTimeMS,
)

results2 = list(result)




In [160]:
embeddig = MistralEmbedder()
messages = embeddig.embed_messages(results2)


🚀 Traitement de 73646 messages par batchs de 250...
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1/73646 : OK
✅ 1

In [161]:
# print(len(messages))
# for msg in messages:
#     print(msg['content']['body'])

In [162]:
import pandas as pd
messages_df = pd.DataFrame([msg['content'] for msg in messages])

In [163]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))

def remove_stopwords_nltk(text):
    words = text.split()
    filtered = [w for w in words if w.lower() not in stop_words]
    return " ".join(filtered)

messages_df['body_clean'] = messages_df['body'].apply(remove_stopwords_nltk)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\p2972\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [164]:
custom_stop_words = [
    # Politesse et formules sociales
    "bonjour", "bonsoir", "salut", "coucou", "hello", "merci", "merci beaucoup", "merci d’avance", "merci par avance",
    "cordialement", "cordial", "bien cordialement", "très cordialement", "sincères salutations",
    "salutations", "amicalement", "bonne journée", "bonne soirée", "bonne continuation", "bonne chance",
    "belle journée", "excellente journée", "aurevoir", "au revoir", "à bientôt", "à plus", "à plus tard",
    "à très vite", "à tous", "à toutes", "bien à vous", "avec mes remerciements", "je vous remercie", "merci de votre retour",

    # Mots de clôture
    "en vous remerciant", "restant à disposition", "je reste à disposition", "restant disponible",
    "dans l’attente", "dans l’attente de votre retour", "dans l’attente de votre réponse",
    "dans l’attente d’une réponse", "dans l’attente de vos nouvelles",

    # Excuses/intro inutiles
    "désolé", "pardon", "excusez-moi", "je m’excuse", "je vous prie", "j’espère que vous allez bien",
    "j’espère que tout va bien", "j’espère que vous passez une bonne journée",

    # Demande
    "svp", "s’il vous plaît", "merci d’avance", "pouvez-vous", "pourriez-vous", "je souhaiterais", "je voudrais", "je souhaite",

    # Petits mots vides
    "bien", "bonne", "super", "ok", "d’accord", "voilà", "juste", "petit", "petite", "simplement", "très", "vraiment",

    # Phrases types forums
    "je suis nouveau", "nouveau sur le forum", "premier message", "post", "message", "topic", "discussion", "sujet",
    "question", "problème", "besoin d’aide", "j’ai un souci", "je rencontre un problème", "à propos", "concernant",

    # Connecteurs/formules vides
    "donc", "alors", "du coup", "en fait", "ainsi", "également", "aussi", "par ailleurs", "de plus", "cependant", "en revanche", "mais", "toutefois",

    # Divers
    "merci!", "merci.", "bonjour!", "bonjour.", "salut!", "salut.", "cordialement.", "cordialement!", "merci beaucoup!", "merci beaucoup.",

    # Smiley et signatures
    ":)", ":(", ":D", ";)", ":P", "😉", "😊", "😃", "👍", "👋", "see you", "best regards", "regards", "thank you",

    # Redondances accidentelles (liées à la casse ou au formatage)
    "merci!", "merci.", "cordialement.", "cordialement!", "bonjour!", "bonjour.", "salut!", "salut.",

    # Mots anglais qui ne servent pas pour des topics
    "hi", "hello", "thanks", "regards"
]


def remove_custom_stopwords(text):
    return " ".join([w for w in text.split() if w.lower() not in custom_stop_words])

messages_df['body_clean'] = messages_df['body_clean'].apply(remove_custom_stopwords)

In [165]:
from bertopic import BERTopic

messages_df['body_clean'].tolist()

["cadre études d'ingénieur j'ai ammené aborder management manière théorique. poste depuis peu plus d'un an maintenant, souhaitais mettre jour connaissances, confronter situations professionnelles j'ai rencontré. J'aimerais terme MOOC disposer compétences manageriales complément expérience technique.",
 "Bonjour, actuellement formation d'assistante dirigeant d'entreprise artisanale, a quelques années j'ai l'opportunité d'effectuer période d'essai tant manager, malheureusement n'avais convaincu. souhaiterai approfondir connaissances, sait pourrai avoir besoin plus tard.",
 "tous (et crois nombreux monde) Archiviste formation, j'enseigne métier depuis 14 ans université française. C'est premier MOOC motivation triple : - découvrir qu'est MOOC, curiosité, peut-être lancer conception moi-même. impliquée TICE depuis nombreuses années inscrite parallèlement Certificat Informatique Internet, niveau 2 Enseignants. - développer connaissances management leadership aider étudiants positionner comme

In [166]:


# Exemple sur les 20 premiers messages
# for txt in messages_df['body_clean'].head(20):
#     print(detect(txt), txt[:60])

from langdetect import detect, LangDetectException

def detect_lang_safe(text):
    try:
        if isinstance(text, str) and text.strip():
            return detect(text)
        else:
            return "NA"
    except LangDetectException:
        return "NA"

messages_df['lang'] = messages_df['body_clean'].apply(detect_lang_safe)

In [167]:
print(messages_df['lang'].value_counts())


lang
fr       48310
en       24004
NA         253
it         196
es         176
ca         131
nl          73
af          67
de          66
ro          53
pt          52
da          39
no          31
id          30
sv          28
et          25
vi          14
cy          12
fi          12
tl          11
so          10
ar           8
hr           7
sk           6
sq           5
sl           4
sw           4
pl           4
lt           3
tr           3
ko           2
hu           2
bn           1
zh-cn        1
cs           1
ru           1
lv           1
Name: count, dtype: int64


In [168]:
messages_df_fr = messages_df[messages_df['lang'] == 'fr']

docs = messages_df_fr['body_clean'].dropna().tolist()


In [169]:
from hdbscan import HDBSCAN

clusterer = HDBSCAN(min_cluster_size=40, min_samples=10)
topic_model = BERTopic(hdbscan_model=clusterer, language="french")
topics, probs = topic_model.fit_transform(docs)






In [172]:
fig = topic_model.visualize_topics(width=1200, height=900)
fig.write_html("bertopic_topics.html")

In [171]:
# from bertopic import BERTopic

# messages_df['body'].tolist()
# docs = messages_df['body'].tolist()
# topic_model = BERTopic()
# topics, probs = topic_model.fit_transform(docs)
# topic_model.visualize_topics()

